## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [Extract findings in radiology reports](https://aws.amazon.com/marketplace/pp/prodview-kriuw433xdo4u)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

### Radiology Entity Detection, Assertion Status Assignment, and Relation Extraction Pipeline

Overview:

The Radiology Entity Detection, Assertion Status Assignment, and Relation Extraction Pipeline is a comprehensive solution designed to enhance the analysis of radiology documents. Its primary focus lies in identifying crucial entities within radiology texts, assigning assertion statuses to these entities, and extracting relevant relations among them.


- **Model**: `en.explain_doc.clinical_radiology.pipeline`
- **Model Description**: A pipeline for detecting posology entities with the `ner_radiology` NER model, assigning their assertion status with `assertion_dl_radiology` model, and extracting relations between posology-related terminology with `re_test_problem_finding` relation extraction model.


In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [5]:
model_name = "en-explain-doc-clinical-radiology-pipeline"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.xlarge"

### A. Create an endpoint

In [6]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

--------!

Once endpoint has been created, you would be able to perform real-time inference.

In [7]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

    input_data = json.dumps(data) if content_type == 'application/json' else data

    # Write input data to file
    with open(input_file_name, 'w') as f:
        f.write(input_data)

    # Upload input data to S3
    s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=bytes(input_data.encode('UTF-8')))

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    # Upload response data to S3
    output_s3_key = f"{s3_output_dir}/{os.path.basename(output_file_name)}"
    if accept == 'application/json':
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=json.dumps(response_data).encode('UTF-8'))
    else:
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=response_data)

    return response_data

### Initial Setup

In [8]:
docs = [
        """Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality  is discernible.""",

        """Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma."""
]

sample_text = """ Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new.  The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine"""

### JSON

#### Example 1

  **Input format**:
  
  
```json
{
    "text": "Single text document"
}
```

In [9]:
input_json_data = {"text": sample_text}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
ner_df = pd.DataFrame(data["ner_predictions"])
assertion_df = pd.DataFrame(data["assertion_predictions"])
relation_df = pd.DataFrame(data["relation_predictions"])

In [10]:
ner_df

,document,ner_chunk,begin,end,ner_label,ner_confidence
0,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,Cardiac size,1,12,ImagingFindings,0.8463
1,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,left,41,44,Direction,0.9433
2,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,pleural effusion,46,61,Disease_Syndrome_Disorder,0.69865
3,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,right,77,81,Direction,0.9554
4,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,effusion,83,90,ImagingFindings,0.9442
5,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,upper lungs,105,115,BodyPart,0.5999
6,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,clear,121,125,ImagingFindings,0.7488
7,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,Right,128,132,Direction,0.9839
8,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,lower lobe opacities,134,153,Symptom,0.30993333
9,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,CT,180,181,Imaging_Test,0.9859


In [11]:
assertion_df

,document,ner_chunk,begin,end,ner_label,assertion
0,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,Cardiac size,1,12,ImagingFindings,Confirmed
1,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,pleural effusion,46,61,Disease_Syndrome_Disorder,Confirmed
2,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,effusion,83,90,ImagingFindings,Confirmed
3,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,clear,121,125,ImagingFindings,Confirmed
4,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,lower lobe opacities,134,153,Symptom,Confirmed
5,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,pneumothorax,196,207,ImagingFindings,Negative
6,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,mild degenerative changes,220,244,ImagingFindings,Confirmed


In [12]:
relation_df

,document,ner_chunk1,ner_chunk1_begin,ner_chunk1_end,ner_label1,ner_chunk2,ner_chunk2_begin,ner_chunk2_end,ner_label2,relation,relation_confidence
0,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,upper lungs,105,115,BodyPart,clear,121,125,ImagingFindings,not_related,0.947544
1,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,mild degenerative changes,220,244,ImagingFindings,thoracic spine,253,266,BodyPart,is_related,0.99999964


#### Example 2

  **Input format**:
  
  
```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ]
}
```

In [13]:
input_json_data = {"text": docs}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
ner_df = pd.DataFrame(data["ner_predictions"])
assertion_df = pd.DataFrame(data["assertion_predictions"])
relation_df = pd.DataFrame(data["relation_predictions"])

In [14]:
ner_df

,document,ner_chunk,begin,end,ner_label,ner_confidence
0,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",bilateral,69,77,Direction,0.9853
1,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",foot pain,79,87,Symptom,0.6934
2,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",traumatic,165,173,Disease_Syndrome_Disorder,0.7773
3,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",arthritic,176,184,Disease_Syndrome_Disorder,0.6663
4,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",developmental abnormality,190,214,ImagingFindings,0.4231
5,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",Bilateral breast,0,15,BodyPart,0.945
6,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",ultrasound,17,26,Imaging_Test,0.6734
7,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",ovoid mass,78,87,ImagingFindings,0.6095
8,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",0.5 x 0.5 x 0.4 cm,113,130,Measurements,0.9908667
9,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",anteromedial aspect of the left shoulder,163,202,BodyPart,0.75051665


In [15]:
assertion_df

,document,ner_chunk,begin,end,ner_label,assertion
0,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",foot pain,79,87,Symptom,Confirmed
1,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",traumatic,165,173,Disease_Syndrome_Disorder,Negative
2,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",arthritic,176,184,Disease_Syndrome_Disorder,Negative
3,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",developmental abnormality,190,214,ImagingFindings,Negative
4,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",ovoid mass,78,87,ImagingFindings,Confirmed
5,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",mass,210,213,ImagingFindings,Confirmed
6,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",isoechoic echotexture,228,248,ImagingFindings,Confirmed
7,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",internal color flow,294,312,ImagingFindings,Negative
8,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",benign fibrous tissue,334,354,ImagingFindings,Suspected
9,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",lipoma,361,366,Disease_Syndrome_Disorder,Suspected


In [16]:
relation_df

,document,ner_chunk1,ner_chunk1_begin,ner_chunk1_end,ner_label1,ner_chunk2,ner_chunk2_begin,ner_chunk2_end,ner_label2,relation,relation_confidence
0,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",Bilateral breast,0,15,BodyPart,ultrasound,17,26,Imaging_Test,is_related,1.0
1,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",Bilateral breast,0,15,BodyPart,ovoid mass,78,87,ImagingFindings,is_related,0.99999714
2,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",ultrasound,17,26,Imaging_Test,ovoid mass,78,87,ImagingFindings,is_related,0.99956936
3,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",ovoid mass,78,87,ImagingFindings,0.5 x 0.5 x 0.4 cm,113,130,Measurements,is_related,1.0
4,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",mass,210,213,ImagingFindings,adjacent muscle,257,271,BodyPart,is_related,0.9976392
5,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",isoechoic echotexture,228,248,ImagingFindings,adjacent muscle,257,271,BodyPart,is_related,0.9999993


### JSON Lines

  **Input format**:
  
```json
{"text": "Text document 1"}
{"text": "Text document 2"}
```

In [17]:
import json

def create_jsonl(records):
    json_records = []

    for text in records:
        record = {
            "text": text
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

input_jsonl_data = create_jsonl(docs)

#### Example 1

In [18]:
data = process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"ner_predictions": [{"ner_chunk": "bilateral", "begin": 69, "end": 77, "ner_label": "Direction", "ner_confidence": "0.9853"}, {"ner_chunk": "foot pain", "begin": 79, "end": 87, "ner_label": "Symptom", "ner_confidence": "0.6934"}, {"ner_chunk": "traumatic", "begin": 165, "end": 173, "ner_label": "Disease_Syndrome_Disorder", "ner_confidence": "0.7773"}, {"ner_chunk": "arthritic", "begin": 176, "end": 184, "ner_label": "Disease_Syndrome_Disorder", "ner_confidence": "0.6663"}, {"ner_chunk": "developmental abnormality", "begin": 190, "end": 214, "ner_label": "ImagingFindings", "ner_confidence": "0.4231"}], "assertion_predictions": [{"ner_chunk": "foot pain", "begin": 79, "end": 87, "ner_label": "Symptom", "assertion": "Confirmed"}, {"ner_chunk": "traumatic", "begin": 165, "end": 173, "ner_label": "Disease_Syndrome_Disorder", "assertion": "Negative"}, {"ner_chunk": "arthritic", "begin": 176, "end": 184, "ner_label": "Disease_Syndrome_Disorder", "assertion": "Negative"}, {"ner_chunk": "devel

### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [19]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Batch inference

In [20]:
import json
import os

input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)

validation_json_file_name = "input.json"

validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"

def write_and_upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    if file_format == ".json":
        input_data = json.dumps(input_data)

    with open(file_name, "w") as f:
        f.write(input_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

In [21]:
input_jsonl_data = create_jsonl(docs)
input_json_data = {"text": docs}

write_and_upload_to_s3(input_json_data, f"{json_input_dir}/{validation_json_file_name}")

write_and_upload_to_s3(input_jsonl_data, f"{jsonl_input_dir}/{validation_jsonl_file_name}")

### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [23]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):

    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    # Define a dictionary with DataFrame objects and their names
    data_frames = {
        "NER Predictions DataFrame": pd.DataFrame(data['ner_predictions']),
        "Assertion Predictions DataFrame": pd.DataFrame(data['assertion_predictions']),
        "Relation Predictions DataFrame": pd.DataFrame(data['relation_predictions'])
    }

    # Display and save each DataFrame
    for name, df in data_frames.items():
        print(name + ":")
        display(df)
    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

In [24]:
process_s3_json_output_and_save(validation_json_file_name)

NER Predictions DataFrame:


,document,ner_chunk,begin,end,ner_label,ner_confidence
0,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",bilateral,69,77,Direction,0.9853
1,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",foot pain,79,87,Symptom,0.6934
2,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",traumatic,165,173,Disease_Syndrome_Disorder,0.7773
3,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",arthritic,176,184,Disease_Syndrome_Disorder,0.6663
4,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",developmental abnormality,190,214,ImagingFindings,0.4231
5,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",Bilateral breast,0,15,BodyPart,0.945
6,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",ultrasound,17,26,Imaging_Test,0.6734
7,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",ovoid mass,78,87,ImagingFindings,0.6095
8,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",0.5 x 0.5 x 0.4 cm,113,130,Measurements,0.9908667
9,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",anteromedial aspect of the left shoulder,163,202,BodyPart,0.75051665


Assertion Predictions DataFrame:


,document,ner_chunk,begin,end,ner_label,assertion
0,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",foot pain,79,87,Symptom,Confirmed
1,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",traumatic,165,173,Disease_Syndrome_Disorder,Negative
2,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",arthritic,176,184,Disease_Syndrome_Disorder,Negative
3,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",developmental abnormality,190,214,ImagingFindings,Negative
4,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",ovoid mass,78,87,ImagingFindings,Confirmed
5,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",mass,210,213,ImagingFindings,Confirmed
6,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",isoechoic echotexture,228,248,ImagingFindings,Confirmed
7,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",internal color flow,294,312,ImagingFindings,Negative
8,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",benign fibrous tissue,334,354,ImagingFindings,Suspected
9,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",lipoma,361,366,Disease_Syndrome_Disorder,Suspected


Relation Predictions DataFrame:


,document,ner_chunk1,ner_chunk1_begin,ner_chunk1_end,ner_label1,ner_chunk2,ner_chunk2_begin,ner_chunk2_end,ner_label2,relation,relation_confidence
0,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",Bilateral breast,0,15,BodyPart,ultrasound,17,26,Imaging_Test,is_related,1.0
1,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",Bilateral breast,0,15,BodyPart,ovoid mass,78,87,ImagingFindings,is_related,0.99999714
2,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",ultrasound,17,26,Imaging_Test,ovoid mass,78,87,ImagingFindings,is_related,0.99956936
3,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",ovoid mass,78,87,ImagingFindings,0.5 x 0.5 x 0.4 cm,113,130,Measurements,is_related,1.0
4,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",mass,210,213,ImagingFindings,adjacent muscle,257,271,BodyPart,is_related,0.9976392
5,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",isoechoic echotexture,228,248,ImagingFindings,adjacent muscle,257,271,BodyPart,is_related,0.9999993


### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [26]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [27]:
process_s3_jsonlines_output_and_save(validation_jsonl_file_name)

{"ner_predictions": [{"ner_chunk": "bilateral", "begin": 69, "end": 77, "ner_label": "Direction", "ner_confidence": "0.9853"}, {"ner_chunk": "foot pain", "begin": 79, "end": 87, "ner_label": "Symptom", "ner_confidence": "0.6934"}, {"ner_chunk": "traumatic", "begin": 165, "end": 173, "ner_label": "Disease_Syndrome_Disorder", "ner_confidence": "0.7773"}, {"ner_chunk": "arthritic", "begin": 176, "end": 184, "ner_label": "Disease_Syndrome_Disorder", "ner_confidence": "0.6663"}, {"ner_chunk": "developmental abnormality", "begin": 190, "end": 214, "ner_label": "ImagingFindings", "ner_confidence": "0.4231"}], "assertion_predictions": [{"ner_chunk": "foot pain", "begin": 79, "end": 87, "ner_label": "Symptom", "assertion": "Confirmed"}, {"ner_chunk": "traumatic", "begin": 165, "end": 173, "ner_label": "Disease_Syndrome_Disorder", "assertion": "Negative"}, {"ner_chunk": "arthritic", "begin": 176, "end": 184, "ner_label": "Disease_Syndrome_Disorder", "assertion": "Negative"}, {"ner_chunk": "devel

In [28]:
model.delete_model()

INFO:sagemaker:Deleting model with name: en-explain-doc-clinical-radiology-pipel-2024-05-09-14-18-18-617


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

